## RF

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.pipeline import make_pipeline

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import *

import pygam

In [2]:
data = pd.read_csv("../diamonds_train.csv")

In [3]:
datatest = pd.read_csv("../diamonds_test.csv")

In [4]:
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.pipeline import make_pipeline

def cleaner(df):

    #data.drop(["id"], axis=1, inplace=True)

    df1 = df[["carat", "depth", "table", "x", "y", "z", "id"]]
    transformer = Normalizer()
    transformer.fit(df1)
    df1 = pd.DataFrame(transformer.transform(df1),columns=df1.columns)

    df2 = df.copy().drop(["carat", "depth", "table", "x", "y", "z"], axis=1)
    df2 = pd.get_dummies(df2)

    df = pd.concat([df1, df2], axis=1, sort=False)
    df.drop("id", axis=1, inplace=True)
    
    correlation_matrix = data.drop("price", axis=1).corr()
    correlated_features = set()

    for i in range(len(correlation_matrix.columns)):
        for j in range(i):
            if abs(correlation_matrix.iloc[i, j]) >= 0.9:
                colname = correlation_matrix.columns[i]
                correlated_features.add(colname)
    correlated_features = ["table", "x", "y", "z"]

    df_train = df.drop(correlated_features, axis=1)

    """X = df_train.drop("price", axis=1)
    y = data_prueba["price"]"""
    
    return df_train

In [5]:
df_train = cleaner(data)
df_train

,carat,depth,price,cut_Fair,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,color_D,color_E,...,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,0.009173,0.723220,3446,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,0.003738,0.733091,732,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
2,0.003626,0.752959,475,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0.012157,0.724721,9552,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
4,0.007833,0.739924,1276,0,0,1,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40340,0.000012,0.001534,1716,0,0,1,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
40341,0.000008,0.001532,781,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
40342,0.000010,0.001524,1123,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
40343,0.000026,0.001517,5651,0,0,1,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0


In [48]:
df_test = cleaner(datatest)

df_test

,carat,depth,table,cut_Fair,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,color_D,color_E,...,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,0.012840,0.726038,0.677013,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0.006003,0.735633,0.670898,0,0,1,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
2,0.023475,0.715828,0.682291,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0.014139,0.701097,0.701097,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0.006609,0.742584,0.660875,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13444,0.000022,0.004463,0.004314,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,1,0,0
13445,0.000041,0.004522,0.004165,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
13446,0.000017,0.004559,0.004313,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,1,0
13447,0.000022,0.004566,0.004313,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0


### Entrenamiento 100%

In [50]:
X_train = df_train.drop("price", axis=1)
y_train = df_train["price"]

In [51]:
rf_best = RandomForestRegressor(n_estimators=600, min_samples_split=2, min_samples_leaf=1,
                               max_features="auto", max_depth=70, bootstrap=True)

rf_best.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=70, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=600, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

### Aplico a test

In [53]:
#rf_best = RandomForestRegressor(n_estimators=600, min_samples_split=2, min_samples_leaf=1,
                               #max_features="auto", max_depth=70, bootstrap=True)
    
X_test = df_test

y_pred = rf_best.predict(X_test)

In [55]:
len(y_pred)

13449

In [56]:
df_test["price"] = y_pred

In [57]:
df_test

,carat,depth,table,cut_Fair,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,color_D,color_E,...,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2,price
0,0.012840,0.726038,0.677013,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,6489.736667
1,0.006003,0.735633,0.670898,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1735.991667
2,0.023475,0.715828,0.682291,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,14146.765000
3,0.014139,0.701097,0.701097,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,6203.273333
4,0.006609,0.742584,0.660875,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1885.310000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13444,0.000022,0.004463,0.004314,0,0,0,1,0,0,1,...,0,0,0,0,0,0,1,0,0,711.440000
13445,0.000041,0.004522,0.004165,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,2252.490000
13446,0.000017,0.004559,0.004313,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,1,0,551.768333
13447,0.000022,0.004566,0.004313,0,0,1,0,0,1,0,...,0,0,0,0,0,0,1,0,0,737.073333


In [66]:
result = df_test.copy().reset_index()[["index", "price"]].rename(columns={"index": "id"})

In [68]:
result.to_csv("../output/rf_result.csv", index=False) ## Resultado : 946.56088

## Nuevo approach

In [1]:
def cleaner2(df):

    #data.drop(["id"], axis=1, inplace=True)

    df1 = df[["carat", "depth", "table", "x", "y", "z", "id"]]
    transformer = Normalizer()
    transformer.fit(df1)
    df1 = pd.DataFrame(transformer.transform(df1),columns=df1.columns)

    df2 = df.copy().drop(["carat", "depth", "table", "x", "y", "z"], axis=1)
    df2 = pd.get_dummies(df2)

    df = pd.concat([df1, df2], axis=1, sort=False)
    df.drop("id", axis=1, inplace=True)
    
    correlation_matrix = data.drop("price", axis=1).corr()
    correlated_features = set()

    for i in range(len(correlation_matrix.columns)):
        for j in range(i):
            if abs(correlation_matrix.iloc[i, j]) >= 0.9:
                colname = correlation_matrix.columns[i]
                correlated_features.add(colname)
    #correlated_features = ["table", "x", "y", "z"]
    

    #df_train = df.drop(correlated_features, axis=1)

    """X = df_train.drop("price", axis=1)
    y = data_prueba["price"]"""
    
    return df

In [10]:
predictors = ['carat', 'y', 'clarity_I1', 'clarity_SI2', 'clarity_VVS1', 'clarity_VVS2']

In [12]:
df = cleaner2(data)

In [13]:
X = df[predictors]
y = df['price']

In [15]:
from sklearn import ensemble
params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}
clf = ensemble.GradientBoostingRegressor()

clf.fit(X, y)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

### Aplico a test

In [65]:
df_test = cleaner2(datatest)


In [53]:
X_test = df_test[predictors]

In [33]:
y_pred = clf.predict(X_test)

In [34]:
df_test["price"] = y_pred

In [35]:
result = df_test.copy().reset_index()[["index", "price"]].rename(columns={"index": "id"})

In [36]:
result.to_csv("../output/GB_RFECV.csv", index=False)

In [37]:
# Resultado: 1938.96793

## Mismo approach con RF

In [50]:
predictors = ['carat', 'y', 'clarity_I1', 'clarity_SI2', 'clarity_VVS1', 'clarity_VVS2']
df = cleaner2(data)
X_train = df[predictors]
y_train = df['price']


In [51]:
rf_best = RandomForestRegressor(n_estimators=600, min_samples_split=2, min_samples_leaf=1,
                               max_features="auto", max_depth=70, bootstrap=True)

rf_best.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=70, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=600, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

### Aplico a test

In [54]:
y_pred = rf_best.predict(X_test)

In [55]:
df_test["price"] = y_pred

In [56]:
result = df_test.copy().reset_index()[["index", "price"]].rename(columns={"index": "id"})

In [58]:
result.to_csv("../output/RF_RFECV.csv", index=False) ## 1315.61017

### Gradient Boost

In [61]:
df_train = cleaner(data)

In [62]:
X_train = df_train.drop("price", axis=1)
y_train = df_train["price"]

In [63]:
from sklearn import ensemble

params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}
clf = ensemble.GradientBoostingRegressor()

clf.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

### Aplico a test

In [66]:
df_test = cleaner(datatest)

X_test = df_test

y_pred = clf.predict(X_test)

In [67]:
df_test["price"] = y_pred

In [68]:
result = df_test.copy().reset_index()[["index", "price"]].rename(columns={"index": "id"})

In [ ]:
result.to_csv("../output/GB_NoRFECV.csv", index=False) ## 1559.66909

### Feature Scaling

In [6]:
X_train = df_train.drop("price", axis=1)
y_train = df_train["price"]

In [13]:
from sklearn.preprocessing import RobustScaler

transformer = RobustScaler()
transformer.fit(X_train)

X_train = transformer.transform(X_train)
"""X_train_transformed = transformed.transform(X_train)
y_pred = logisticReg.predict(X_train_transformed)"""

'X_train_transformed = transformed.transform(X_train)\ny_pred = logisticReg.predict(X_train_transformed)'

In [14]:
rf_best = RandomForestRegressor(n_estimators=600, min_samples_split=2, min_samples_leaf=1,
                               max_features="auto", max_depth=70, bootstrap=True)

rf_best.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=70, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=600, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [15]:

df_test = cleaner(datatest)

X_test = df_test

X_test_transformed = transformer.transform(X_test)
y_pred = rf_best.predict(X_test_transformed)

In [16]:
df_test["price"] = y_pred
result = df_test.copy().reset_index()[["index", "price"]].rename(columns={"index": "id"})
result.to_csv("../output/RF_StandardScaler.csv", index=False) ## 925.36454

In [18]:
## Última oportunidad

In [19]:
data2 = pd.read_csv("../output/clean_diamonds.csv")

In [20]:
data = pd.read_csv("../output/clean_diamonds.csv")
data.drop(['id'], axis=1, inplace=True)


In [21]:
correlation_matrix = data.drop("price", axis=1).corr()
correlated_features = set()

for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.8:
            colname = correlation_matrix.columns[i]
            correlated_features.add(colname)

correlated_features 
dataCut = data.copy()

dataCut.drop(correlated_features, axis=1, inplace=True)
X = dataCut.drop("price", axis=1)
y = dataCut["price"]